In [37]:
!pip install -q langchain langchain-community langchain-huggingface huggingface-hub youtube-transcript-api faiss-cpu

# load youtube transcript


In [38]:
from youtube_transcript_api import YouTubeTranscriptApi
def get_transcript(yt_video_id):
  ytApi = YouTubeTranscriptApi()
  Transcripts  = ytApi.fetch(yt_video_id)
  Transcript = ' '.join([item.text for item in Transcripts.snippets])
  return Transcript

  return Transcript

In [39]:
transcript = get_transcript('Gfr50f6ZBvo')


# Text Splitter using 

In [40]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFacePipeline ,HuggingFaceEmbeddings ,ChatHuggingFace
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate


def create_chunk(transcript):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    doc = text_splitter.split_text(transcript)
    return doc

In [41]:
docs = create_chunk(transcript)

In [42]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
llm=HuggingFacePipeline.from_model_id(model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",task="text-generation")
model = ChatHuggingFace(llm=llm)

Device set to use cuda:0


# vectore store 


In [43]:
vectorstore = FAISS.from_texts(docs, embedding=embedding_model)
vector_retriever = vectorstore.as_retriever( search_type='mmr', search_kwargs={"k": 5,'lambda_mult': 0.6})

# Lexical Search


In [44]:
!pip install rank-bm25

In [45]:
from langchain_community.retrievers import BM25Retriever

bm25 = BM25Retriever.from_texts(docs)

# REG FUSION ->lexical search and semantic search 
**combine the two retriever doc and rerank using reciprocal rank algorithom** 
> **reciprocal rank algorithom**

> **rerank[i] +=1/rank+k**

**or**
> **rerank[i]= alpa * vector + (1-alpa) bm25**

In [46]:
from langchain.retrievers import EnsembleRetriever

hybrid_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, bm25],
    weights=[0.5, 0.5]
)

# Query Transformtion

**Step back Query techniques**

**privious model does not give the proper result**


In [47]:

from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
llm2 = HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2',temperature=0.7)
model2 = ChatHuggingFace(llm=llm )

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


class pydantic_structure(BaseModel):
    orginal_query: str = Field(description="The original query provided by the user.")
    Step_back_query: str = Field(description="A broader, more general query to retrieve background information.")


pyparser = PydanticOutputParser(pydantic_object=pydantic_structure)


step_back_query_prompt = PromptTemplate(
    template="""You are an AI assistant tasked with generating a broader, more general version of a user query.
{format_instructions}

Original query: {original_query}""",
    input_variables=['original_query'],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
step_back_chain = step_back_query_prompt | model2 | pyparser

result = step_back_chain.invoke({"original_query": "The Future of Agents"})

print(result)


In [ ]:
step_back_query

# MultiQueryRetriever

**Generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents.**

In [49]:
from langchain.retrievers.multi_query import MultiQueryRetriever

Multi_query_retriever = multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=hybrid_retriever,
    llm=model2  
)

# Rerank using Cross encoder 

In [50]:
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

cross_encoder_reranker = CrossEncoderReranker(
    model=HuggingFaceCrossEncoder(model_name='cross-encoder/ms-marco-MiniLM-L-6-v2'),
)

In [51]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=cross_encoder_reranker, base_retriever=Multi_query_retriever
)

In [52]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [53]:
from langchain_core.output_parsers import StrOutputParser
strParser=StrOutputParser()

In [54]:
def format_docs(docs):
    context = '\n\n'.join(doc.page_content for doc in docs)
    return context

In [55]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
context_chain = RunnableParallel({
    'context':compression_retriever | RunnableLambda(format_docs),
    'question':RunnablePassthrough()
})

In [56]:
yt_chain = context_chain | prompt | model2 | strParser

In [57]:
user_query = input("Enter your Query")

Enter your Query Can you summarize the video


In [58]:
reslut = yt_chain.invoke(user_query)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [59]:
reslut

"<|user|>\n\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      running those those complicated simulations yeah and learning the task of mapping from the initial conditions and the parameters of the simulation learning what the functional would be yeah so it's pretty tricky and we've done it with um you know the nice thing is we there are we can run a lot of the simulations that the molecular dynamics simulations on our compute clusters and so that generates a lot of data so in this case the data is generated so we like those sort of systems and that's why we use games simulator generated data and we can kind of create as much of it as we want really um and just let's leave some you know if any computers are free in the cloud we just run we run some of these calculations right compute cluster calculation that's all the the free compute times used up on quantum mechanics qu